### Import Required Libraries and Set Up Environment Variables

In [30]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [31]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [32]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
nyt_query_url = f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}"
print(nyt_query_url)



https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&api-key=ipnXdj9Apg5OBskiC2cgTfoT7rw1xsFm


In [33]:
# Create an empty list to store the reviews
reviews_list =  []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    nyt_page_url = f"{nyt_query_url}&page={page}"

    try:    
    # Make a "GET" request and retrieve the JSON
        nyt_response = requests.get(nyt_page_url)
        nyt_response_json = nyt_response.json()

        # Checking if there are reviews in the response
        if nyt_response_json["response"]["docs"]:

            # loop through the reviews["response"]["docs"] and append each review to the list
            for review in nyt_response_json["response"]["docs"]:
                reviews_list.append(review)

            # Print the page that was just retrieved
            print(f"Retrieved page {page}")
        else:
            # Print the page number that had no results then break from the loop
            print(f"No results on page {page}")
            break
    except Exception as e:
        print(f"An error occured: {e}")
        break
            
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)


Retrieved page 0
Retrieved page 1
Retrieved page 2
Retrieved page 3
Retrieved page 4
Retrieved page 5
Retrieved page 6
Retrieved page 7
Retrieved page 8
Retrieved page 9
Retrieved page 10
Retrieved page 11
Retrieved page 12
Retrieved page 13
Retrieved page 14
Retrieved page 15
Retrieved page 16
Retrieved page 17
Retrieved page 18
Retrieved page 19


In [34]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for review in reviews_list[:5]:
    print(json.dumps(review, indent=4))

{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
            "rank": 3,
            "major": "N"
        }
    ],
  

In [35]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
review_df = pd.json_normalize(reviews_list)
print(review_df)

                                               web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/what...   
2    https://www.nytimes.com/2023/05/04/movies/you-...   
3    https://www.nytimes.com/2023/04/21/movies/a-to...   
4    https://www.nytimes.com/2023/04/20/movies/othe...   
..                                                 ...   
195  https://www.nytimes.com/2017/03/09/movies/the-...   
196  https://www.nytimes.com/2017/03/09/movies/revi...   
197  https://www.nytimes.com/2017/03/02/movies/love...   
198  https://www.nytimes.com/2017/02/16/movies/ever...   
199  https://www.nytimes.com/2017/02/09/movies/kedi...   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Two childhood friends navigate cultural differ...  The New York Times   
2    Religion comes between two girls falling in lo...  The New York 

In [36]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

def extract_title(headline_main):
    # Check if headline starts with "Review:"
    if headline_main.startswith("Review:"):
        # Extract the title after "Review:"
        return headline_main.split("Review:")[1].strip()

    # Original pattern extraction
    start = headline_main.find("\u2018") + 1
    end = headline_main.find("\u2019 Review")
    if start > 0 and end > start:
        return headline_main[start:end]
    else:
        return "Title Not Found"

# Apply the function to the DataFrame
review_df['title'] = review_df['headline.main'].apply(extract_title)


In [37]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
review_df['keywords'] = review_df['keywords'].apply(extract_keywords)

# Print the first 5 rows of the 'keywords' column
print(review_df['keywords'].head(5))


0    subject: Movies;creative_works: The Attachment...
1    subject: Movies;persons: Kapur, Shekhar;person...
2    subject: Movies;creative_works: You Can Live F...
3    subject: Movies;creative_works: A Tourist's Gu...
4    subject: Movies;persons: Zlotowski, Rebecca;cr...
Name: keywords, dtype: object


In [38]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = review_df['title'].to_list()
len(titles_list)

200

### Access The Movie Database API

In [39]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

tmdb_query = f'{url}{tmdb_key_string}'
print(tmdb_query)

https://api.themoviedb.org/3/search/movie?query=&api_key=a3d97bea5c6285f85d9529680c3afb4f


In [40]:
# Create an empty list to store the results
tmdb_results = []


# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        print("We're just taking a quick break")
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1

    
    # Perform a "GET" request for The Movie Database
    tmdb_search_url = f'{url}{title}{tmdb_key_string}'

    # Include a try clause to search for the full movie details.
    try:
        tmdb_search_response = requests.get(tmdb_search_url)
        tmdb_search_results = tmdb_search_response.json()
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id
        if tmdb_search_results['results']:
            movie_id = tmdb_search_results['results'][0]['id']

        # Make a request for a the full movie details
            details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
            

        # Execute "GET" request with url
            details_response = requests.get(details_url)
            movie_details = details_response.json()
        
        # Extract the genre names into a list
            genres = [genre['name'] for genre in movie_details.get('genres', [])]

        # Extract the spoken_languages' English name into a list
            languages = [lang['english_name'] for lang in movie_details.get('spoken_languages', [])]

        # Extract the production_countries' name into a list
            countries = [country['name'] for country in movie_details.get('production_countries', [])]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
            movie_data = {
                'title': title,
                'genres': genres,
                'languages': languages,
                'countries': countries,
                # Add other fields if needed
            }
            tmdb_results.append(movie_data)
        # Print out the title that was found
            print(f"Found details for movie: {title}")

        else:
            # No movie found for this title
            print(f"No details found for movie: {title}")
    except Exception as e:
        print(f"An error occurred while processing {title}: {e}")


Found details for movie: The Attachment Diaries
No details found for movie: ‘What’s Love Got to Do With It?’ Probably a Lot
Found details for movie: You Can Live Forever
Found details for movie: A Tourist’s Guide to Love
Found details for movie: Other People’s Children
Found details for movie: One True Loves
Found details for movie: The Lost Weekend: A Love Story
Found details for movie: A Thousand and One
Found details for movie: Your Place or Mine
Found details for movie: Love in the Time of Fentanyl
Found details for movie: Pamela, a Love Story
Found details for movie: In From the Side
Found details for movie: After Love
Found details for movie: Alcarràs
Found details for movie: Nelly & Nadine
Found details for movie: Lady Chatterley’s Lover
Found details for movie: The Sound of Christmas
Found details for movie: The Inspection
Found details for movie: Bones and All
Found details for movie: My Policeman
Found details for movie: About Fate
Found details for movie: Waiting for Bojangl

In [41]:
print(len(tmdb_results))

141


In [42]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for movie in tmdb_results[:5]:
    print(json.dumps(movie, indent=4))

{
    "title": "The Attachment Diaries",
    "genres": [
        "Drama",
        "Mystery",
        "Thriller",
        "Horror"
    ],
    "languages": [
        "Spanish"
    ],
    "countries": [
        "Argentina"
    ]
}
{
    "title": "You Can Live Forever",
    "genres": [
        "Drama",
        "Romance"
    ],
    "languages": [
        "English",
        "French"
    ],
    "countries": [
        "Canada"
    ]
}
{
    "title": "A Tourist\u2019s Guide to Love",
    "genres": [
        "Romance",
        "Comedy"
    ],
    "languages": [
        "English",
        "Vietnamese"
    ],
    "countries": [
        "United States of America"
    ]
}
{
    "title": "Other People\u2019s Children",
    "genres": [
        "Drama",
        "Comedy"
    ],
    "languages": [
        "English",
        "French"
    ],
    "countries": [
        "France"
    ]
}
{
    "title": "One True Loves",
    "genres": [
        "Romance",
        "Drama",
        "Comedy"
    ],
    "languages

In [43]:
# Convert the results to a DataFrame
movie_details_df = pd.json_normalize(tmdb_results)
print(movie_details_df.head(5))

                       title                              genres  \
0     The Attachment Diaries  [Drama, Mystery, Thriller, Horror]   
1       You Can Live Forever                    [Drama, Romance]   
2  A Tourist’s Guide to Love                   [Romance, Comedy]   
3    Other People’s Children                     [Drama, Comedy]   
4             One True Loves            [Romance, Drama, Comedy]   

               languages                            countries  
0              [Spanish]                          [Argentina]  
1      [English, French]                             [Canada]  
2  [English, Vietnamese]           [United States of America]  
3      [English, French]                             [France]  
4              [English]  [Germany, United States of America]  


### Merge and Clean the Data for Export

In [44]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(review_df, movie_details_df, on='title', how='inner')

In [45]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'languages', 'countries']

# Create a list of characters to remove
character_to_remove = ["[", "]", "'", "\""]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for char in character_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, '', regex=False)

# Display the fixed DataFrame
print(merged_df.head())

                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/you-...   
2  https://www.nytimes.com/2023/04/21/movies/a-to...   
3  https://www.nytimes.com/2023/04/20/movies/othe...   
4  https://www.nytimes.com/2023/04/13/movies/one-...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Religion comes between two girls falling in lo...  The New York Times   
2  Rachael Leigh Cook stars in this bland rom-com...  The New York Times   
3  A radiant Virginie Efira stars as a Parisian t...  The New York Times   
4  A film adaptation of Taylor Jenkins Reid’s nov...  The New York Times   

                                            keywords  \
0  subject: Movies;creative_works: The Attachment...   
1  subject: Movies;creative_works: You Can Live F...   
2  subject: Movies;creative_works: A T

In [46]:
# Drop "byline.person" column
del merged_df['byline.person']

In [47]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)

In [48]:
# Export data to CSV without the index
merged_df.to_csv('my_data.csv', index=False)